In [1]:
import os
import json

from dotenv import load_dotenv
from docling.document_converter import DocumentConverter

from schemas.pydantic_schema import DispositifAide
from schemas.schema_utils import pydantic_to_json_schema

/Users/theo.moreau/Documents/betagouv/aides-agri/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
API_KEY = os.getenv("ALBERT_API_KEY")
ENDPOINT = "https://albert.api.etalab.gouv.fr/v1/chat/completions"
JSON_SCHEMA = pydantic_to_json_schema(model=DispositifAide, schema_name="aide-agri")

In [3]:
document_path = [
  "https://aides-redevances.eau-loire-bretagne.fr/files/live/sites/aides-redevances/files/Aides-12prog/Fiches-actions/AGRI_1.pdf",
  "./2017-561_final.pdf"
]

In [4]:
def get_document_context(file_path):
  converter = DocumentConverter()
  doc = converter.convert(file_path).document
  return doc.export_to_text()

In [5]:
import requests

def get_header():
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {API_KEY}",  # Remplace par ta vraie clé
        "Content-Type": "application/json",
    }
    return headers

def get_body_text(model_name, user_message, **kwargs):
    payload = {
        "model": model_name,
        "messages": [
            {
                "role": "user",
                "content": user_message
            }
        ],
        "response_format": {
            "type": "text"
        },
        **kwargs
    }

    return payload

def get_body(model_name, instruction_prompt, user_message, json_schema, **kwargs):
    payload = {
        "model": model_name,
        "messages": [
            {
                "role": "system",
                "content": instruction_prompt
            },
            {
                "role": "user",
                "content": user_message
            }
        ],
        "response_format": {
            "type": "json_schema",
            "json_schema": json_schema
        },
        **kwargs
    }

    return payload

def albert_structured_output(model_name, instruction_prompt, user_message, json_schema, **kwargs):
    url = ENDPOINT
    headers = get_header()
    body = get_body(model_name, instruction_prompt, user_message, json_schema, **kwargs)

    response = requests.post(url, headers=headers, json=body)

    # Vérification du statut et affichage
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erreur {response.status_code}: {response.text}")

In [6]:
INSTRUCTION_PROMPT = """
Tu es un modèle de langage spécialisé dans l’analyse et la synthèse d’informations.
Ta mission est de produire un résultat strictement structuré en JSON, conforme au schéma spécifié ci-dessous.
- Tu dois suivre fidèlement la structure : aucune information hors du format demandé.
- Ne mets aucun commentaire, texte explicatif ou balise en dehors du JSON.
- Les valeurs doivent respecter le type indiqué (string, integer, boolean, etc.).
- Si une donnée est manquante, utilise null plutôt qu’un texte libre.
"""

In [7]:
responses = []

for idx, path in enumerate(document_path):
  print(f"Analyzing document {idx}")
  document_context = get_document_context(path)
  full_response = albert_structured_output("albert-small", INSTRUCTION_PROMPT, document_context, JSON_SCHEMA, temperature=1, max_completion_tokens=400)
  responses.append(full_response)

Analyzing document 0


2025-09-18 12:30:38,654 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-09-18 12:30:38,735 - INFO - Going to convert document batch...
2025-09-18 12:30:38,735 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-09-18 12:30:38,750 - INFO - Loading plugin 'docling_defaults'
2025-09-18 12:30:38,752 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-09-18 12:30:38,764 - INFO - Loading plugin 'docling_defaults'
2025-09-18 12:30:38,767 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-09-18 12:30:38,933 - INFO - Accelerator device: 'mps'
2025-09-18 12:30:40,623 - INFO - Accelerator device: 'mps'
2025-09-18 12:30:41,490 - INFO - Accelerator device: 'mps'
2025-09-18 12:30:41,939 - INFO - Processing document AGRI_1.pdf
2025-09-18 12:30:45,532 - INFO - Finished converting document AGRI_1.pdf in 7.33 sec.
2025-09-18 12:30:45,547 - WARNING - Parameter `strict_text` has b

Analyzing document 1


2025-09-18 12:30:54,946 - INFO - Accelerator device: 'mps'
2025-09-18 12:30:55,742 - INFO - Accelerator device: 'mps'
2025-09-18 12:30:56,106 - INFO - Processing document 2017-561_final.pdf
2025-09-18 12:31:01,304 - INFO - Finished converting document 2017-561_final.pdf in 7.93 sec.
2025-09-18 12:31:01,314 - WARNING - Parameter `strict_text` has been deprecated and will be ignored.


In [8]:
print(responses[0]["choices"][0]["message"]["content"])

{
  "titre": "Accompagnement collectif et individuel des agriculteurs",
  "description": "Cette aide vise à soutenir l'accompagnement collectif et individuel des agriculteurs pour les aider à adapter leurs pratiques et à adopter des systèmes agricoles plus durables.",
  "eligibilite": [
    "Public ou privé"
  ],
  "types_aides": [
    "financement"
  ],
  "porteurs": [
    {
      "nom": "L'agence de l'eau",
      "roles": [
        "financeur",
        "diffuseur"
      ]
    },
    {
      "nom": "Les comités de pilotages territoriaux"
    ,"roles": [
        "diffuseur"
      ]
    },
    {
      "nom": "Les porteurs de projets Ecophyto"
    ,"roles": [
        "diffuseur"
      ]
    }
  ],
  "cibles": [
    "Agriculteurs"
  ],
  "eligibilite_geographique": "Territoires prioritaires identifiés par le Sdage en vigueur",
  "eligibilite_geographique_exclusions": "Établissements publics",
  "date_mise_a_jour": "2023-12-01T12:00:00Z",
  "conditions_particulieres": [
    {
      "nom": 

In [10]:
print(responses[1]["choices"][0]["message"]["content"])

{
  "titre": "Instructions DGPE/SDC/2017-561",
  "description": "Instructions détaillées sur la procédure pour la mise en œuvre de l'aide à la réinsertion professionnelle",
  "eligibilite": [
    "I. Conditions d'éligibilité de l'exploitation",
    "II. Conditions d'éligibilité du demandeur",
    "III. Avantages liés à l'aide à la réinsertion professionnelle",
    "IV. Dépôt et instruction des demandes",
    "V. Contrôle et suivi des dossiers"
  ],
  "types_aides": [
    "assistance",
    "formation",
    "avantage fiscal",
    "avantage fiscal"
  ],
  "porteurs": [
    {
      "nom": "Hervé DURAND",
      "roles": ["diffuseur", "diffuseur"]
    }
  ],
  "cibles": [
    "Direction générale de la performance économique et environnementale (DGPE)",
    "Sous-direction de la compétitivité (SDC)",
    "Bureau de gestion des risques",
    "Service EPE",
    "Chambre d'Agriculture"
  ],
  "eligibilite_geographique": "France",
  "date_mise_a_jour": "2709-04-19T02:23:31.766Z"
}


In [11]:
for idx, res in enumerate(responses):
  print(f"Token number for response {idx}: {res["usage"]["prompt_tokens"]}")

Token number for response 0: 2057
Token number for response 1: 7789
